# Automation of Multiple Experiments

**This is an archived file. It has now been succeeded by the "[MAIN] Sniffing Procedures" file.**

After the gas injection system was setup, this script was created to test out different experimental procedures and sniffing recipes. It served as a general testing environment for different recipes. After rigorous testing, certain recipes were selected to be official sniffing procedures which are now stored in the sqlite database under the "procedures" table.

In [ ]:
from seabreeze.spectrometers import Spectrometer
from matplotlib import pyplot as plt
import time
import numpy as np
import serial
import sys
import numpy as np
import csv
from scan_functions import * # functions built for this project
import pandas as pd
import sqlite3
import datetime
from scipy.signal import savgol_filter as sgf
from math import factorial as fact
from tkinter import Tk, filedialog
from scipy.signal import find_peaks as peaks
from IPython.display import clear_output

## INITIALIZE COMMUNICATIONS
try: 
    spec = Spectrometer.from_first_available() # Start communication with spectrometer
except Exception as e: 
    pass

time_in_ms = int(input("Set integration time (in ms): "))*10**3
spec.integration_time_micros(time_in_ms) # Sets integration time
conn = sqlite3.connect('Database/Sniffing-Sensor.db') # connect to the database
c = conn.cursor() # Creates a cursor to interact with database
port = input("Which port number is is the teensy?: ").strip()
ser = serial.Serial("COM{}".format(port), 9600) # Opening serial port to communicate with teensy

## ADDITIONAL INFORMATION
output_name = input("What do you want to call the output excel file?: ").strip() # User input for output file

### Additional Testing Procedures

In [ ]:
# Indexing starts at zero
print('Purging for 10 minutes - no measurement')
ser.write(b'A')
time.sleep(600)
ser.write(b'a')

purge = 600

print('Experiment 1 in progress: N for 15, EtOH for 15')
for run in range(1): 
       
    # Experiment Name - Integer ID
    exp_name = "Default"
    c.execute("SELECT ExperimentID FROM Experiments")
    data = c.fetchall()
    if data == []: 
        exp_name = 1
    else:
        exp_name = data[-1][0] + 1

    # Timestamp
    timestamp = str(datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))

    # Compound name coded into its OdorID from the database
    compound = "Ethanol"
    temp = c.execute("SELECT OdorID FROM Odors WHERE Name='%s'" %str(compound.upper())) # All database entries are uppercase
    OdorID = c.fetchall()[0][0]
    print("Run Number: {}\nExperiment Name: {}\nCompound: {}".format(run+1, exp_name, compound))


    # Procedure name coded into its ProcedureID from the database
    temp = c.execute("SELECT ProcedureID FROM Procedures WHERE Name='Testing'")
    ProcedureID = c.fetchall()[0][0]

    # Creates a PNG and CSV File pointing to the location of the files
    PNG = "../Scan CSV Files/{}.png".format(exp_name)
    CSV = "../Scan CSV Files/PNG/{}.png".format(exp_name)


    # ======================= WRITING TO CSV File =======================

    df = pd.DataFrame() # creates a new dataframe for csv file
    df["Wavelengths"] = spec.wavelengths()
    df.set_index("Wavelengths")

    # ============================== RECIPE ==============================

    intensities = [spec.intensities()] # empty list to be populated with intensity values from spec
    elapse = [0] #empty list to be populated with time values

    # Nitrogen flush (15 min) 
    
    ser.write(b'A')
    
    last_time = elapse[-1]
    i = 0
    start_time = time.time()

    while i < 900:
        intensities.append(spec.intensities())
        i = time.time() - start_time
        elapse.append(i+last_time)
        time.sleep(1)
  
    ser.write(b'a')
    
    # Ethanol flush (15 min)

    ser.write(b'B')
    last_time = elapse[-1]
    i = 0
    start_time = time.time()
    
    while i < 900:
        intensities.append(spec.intensities())
        i = time.time() - start_time
        elapse.append(i+last_time)
        if i < 300:
            time.sleep(1/20)
        else: 
            time.sleep(1)
    
    ser.write(b'b')
    
    
    for i in range(len(elapse)):
        df["{}".format(round(elapse[i],2))] = intensities[i] # Write to the i'th column the intensities

    sys.stdout.write("\n{}\nDone Captures!\n".format('-'*30))

    # ========================== WRITING TO FILES ==========================
    print("Saving to CSV in ../Scan CSV Files/{}.csv".format(exp_name))
    df.to_csv("../Scan CSV Files/{}.csv".format(exp_name)) # Writing to the CSV file

    plot_river(np.array(intensities),plt, np) # Plotting river plot
    plt.savefig("../Scan CSV Files/PNG/{}.png".format(exp_name)) # Save figure to file
    plt.close()


    Notes = "Long ethanol scan - 15 min Nitrogen purge followed by 15 min ethanol"

    c.execute("""INSERT INTO Experiments (Timestamp, OdorID, ProcedureID, PNG, CSV, Notes)
                 VALUES (?, ?, ?, ?, ?, ?)""",
                 (timestamp, OdorID, ProcedureID, PNG, CSV, Notes))
    
    conn.commit() # Save values into SQL

    # ========================== DATA ANALYSIS ==========================
    print("Analyzing data and saving plots.")
    # Read file and convert it to pandas dataframe
    df = pd.read_csv("../Scan CSV Files/{}.csv".format(exp_name))
    df.drop("Unnamed: 0", axis=1, inplace=True)
    df.set_index("Wavelengths", inplace=True)
    exp_name = compound + "({})".format(exp_name)
    
    
    ## FOURIER TRANSFORM
    fig = plt.figure(figsize=(14,7))
    ft = fig.add_subplot(111, title="{}: Fourier transform".format(exp_name), xlabel="time (s)")

    sliced = [df.index[500], df.index[1200]] # Wavelengths between 400 and 900

    # taking the transpose means you get the fourier transform with time axis
    ft_data = np.fft.fft( (df.loc[sliced[0]:sliced[1], :]).transpose() ) 

    ft.plot(np.real(ft_data[:,1]))
    ft.plot(np.imag(ft_data[:,1]))

    # axes titles
    ft.set_xticks(np.linspace(0, len(df.columns), 11)) # 11 ticks
    ft.set_xticklabels([df.columns[i] for i in range(len(df.columns)) if i%(int(len(df.columns)/10)) == 0]) 
    plt.savefig("../Analysis Images/{}_fourier.png".format(exp_name))
    plt.close(fig)
    
    ## PHASE CALCULATION
    R = np.real(ft_data[:, 1])
    I = np.imag(ft_data[:, 1])

    # Calculating Phase
    phase = I / (R ** 2 + I ** 2) ** 0.5
    phase = sgf(phase, window_length=31, polyorder=3)

    # Normalization
    phase = phase/np.sqrt(np.sum(phase**2))

    # Plotting
    fig = plt.figure(figsize=(14,7))

    ph = fig.add_subplot(111, title="{}: Phase".format(exp_name), xlabel="time (s)")

    ph.plot(phase)

    ph.set_xticks(np.linspace(0, len(df.columns), 11))
    ph.set_xticklabels([df.columns[i] for i in range(len(df.columns)) if i%int(len(df.columns)/10) == 0])

    plt.savefig("../Analysis Images/{}_phase.png".format(exp_name))
    plt.close(fig)
    
    ## PHASE DERIVATIVE CALCULATION
    phase_deriv = np.diff(phase)
    phase_deriv = sgf(phase_deriv, window_length=31, polyorder=3)

    # Normalization
    phase_deriv = phase_deriv/np.sqrt(np.sum(phase_deriv**2))

    # Plotting
    fig = plt.figure(figsize=(14,7))
    pder = fig.add_subplot(111, title="{} Experiment Phase Derivative".format(exp_name), xlabel="time (s)")

    pder.plot(phase_deriv)

    pder.set_xticks(np.linspace(0, len(df.columns), 11))
    pder.set_xticklabels([df.columns[i] for i in range(len(df.columns)) if i%int(len(df.columns)/10) == 0])

    plt.savefig("../Analysis Images/{}_phase_deriv.png".format(exp_name))
    plt.close(fig)




### Experiment 2

In [2]:
# print('Purging for 10 minutes - no measurement')
# ser.write(b'A')
# time.sleep(600)
# ser.write(b'a')

# Read excel and convert to pandas dataframe
experiment = pd.read_excel("../Experiment Files/Experiment 6 - nonpolar.xlsx")
experiment.set_index("True Runs", inplace=True) # indexed by the True run number
experiment.dropna(axis=0, how="all", inplace=True) #removes all NaN rows

num_exp = len(experiment.index) # create an entry for each experiment run

# Indexing starts at zero
for run in range(num_exp): 
    clear_output() # clears output after every run
    
    # --------------- DESIGNING RECIPE -------------------
    
    o_time = list(experiment['Odorant Time'])[run]
    odorant_time = [o_time for i in range(3)]
    
    purge = int(list(experiment['Purge time'])[run]) # by default dataframes are floats
    
    c_time = list(experiment['Sleep Time'])[run]
    clean_time = [purge] + [c_time for i in range(2)]
    
    delay_time = 20 # scans per second

    
    # --------------- PERFORMING RECIPE -------------------
    
    # Experiment Name - Integer ID
    exp_name = "Default"
    c.execute("SELECT ExperimentID FROM Experiments")
    data = c.fetchall()
    if data == []: 
        exp_name = 1
    else:
        exp_name = data[-1][0] + 1

    # Timestamp
    timestamp = str(datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))

    # Compound name coded into its OdorID from the database
    compound = list(experiment['Current'])[run]
    temp = c.execute("SELECT OdorID FROM Odors WHERE Name='%s'" %str(compound.upper())) # All database entries are uppercase
    OdorID = c.fetchall()[0][0]
    print("Experiment 2 in progress: Run {}\nExperiment Name: {}\nCompound: {}".format(run+12, exp_name, compound))


    # Procedure name coded into its ProcedureID from the database
    temp = c.execute("SELECT ProcedureID FROM Procedures WHERE Name='Testing'")
    ProcedureID = c.fetchall()[0][0]

    # Creates a PNG and CSV File pointing to the location of the files
    PNG = "../Scan CSV Files/{}.png".format(exp_name)
    CSV = "../Scan CSV Files/PNG/{}.png".format(exp_name)


    # ======================= WRITING TO CSV File =======================

    df = pd.DataFrame() # creates a new dataframe for csv file
    df["Wavelengths"] = spec.wavelengths()
    df.set_index("Wavelengths")

    # ============================== RECIPE ==============================

    intensities = [spec.intensities()] # empty list to be populated with intensity values from spec
    elapse = [0] #empty list to be populated with time values

    compound_dict = {"Nitrogen": Nitrogen, 
                     "Water": valveC, 
                     "IPA": valveD, 
                     "Ethanol": valveB,
                     "Pentane": valveC,
                     "Hexane": valveD, 
                     "Heptane": valveB} # Assigns the compound to its injection function
    
    print("Performing {} injection".format(compound))
    for i in range(len(odorant_time)):
        nitro_arg = [clean_time[i], ser, intensities, spec, elapse]
        Nitrogen(*nitro_arg)

        odorant_arg = [odorant_time[i], ser, intensities, spec, delay_time, elapse]
        compound_dict["{}".format(compound)](*odorant_arg) # Change the function based on what odorant you are testing
    
    for i in range(len(elapse)):
        df["{}".format(round(elapse[i],2))] = intensities[i] # Write to the i'th column the intensities

    sys.stdout.write("\n{}\nDone Captures!\n".format('-'*30))

    # ========================== WRITING TO FILES ==========================
    print("Saving to CSV in ../Scan CSV Files/{}.csv".format(exp_name))
    df.to_csv("../Scan CSV Files/{}.csv".format(exp_name)) # Writing to the CSV file

    plot_river(np.array(intensities),plt, np) # Plotting river plot
    plt.savefig("../Scan CSV Files/PNG/{}.png".format(exp_name)) # Save figure to file
    plt.close()


    Notes = "Testing automation."
    Notes += "\nOdorant Times: {}\nClean Times: {}\n".format(", ".join([str(i) for i in clean_time]),
                                                             ", ".join([str(i) for i in odorant_time]))

    c.execute("""INSERT INTO Experiments (Timestamp, OdorID, ProcedureID, PNG, CSV, Notes)
                 VALUES (?, ?, ?, ?, ?, ?)""",
                 (timestamp, OdorID, ProcedureID, PNG, CSV, Notes))
    
    conn.commit() # Save values into SQL

    # ========================== DATA ANALYSIS ==========================
    print("Analyzing data and saving plots.")
    # Read file and convert it to pandas dataframe
    df = pd.read_csv("../Scan CSV Files/{}.csv".format(exp_name))
    df.drop("Unnamed: 0", axis=1, inplace=True)
    df.set_index("Wavelengths", inplace=True)
    exp_name = compound + "({})".format(exp_name)
    
    
    ## FOURIER TRANSFORM
    fig = plt.figure(figsize=(14,7))
    ft = fig.add_subplot(111, title="{}: Fourier transform".format(exp_name), xlabel="time (s)")

    sliced = [df.index[500], df.index[1200]] # Wavelengths between 400 and 900

    # taking the transpose means you get the fourier transform with time axis
    ft_data = np.fft.fft( (df.loc[sliced[0]:sliced[1], :]).transpose() ) 

    ft.plot(np.real(ft_data[:,1]))
    ft.plot(np.imag(ft_data[:,1]))

    # axes titles
    ft.set_xticks(np.linspace(0, len(df.columns), 11)) # 11 ticks
    ft.set_xticklabels([df.columns[i] for i in range(len(df.columns)) if i%(int(len(df.columns)/10)) == 0]) 
    plt.savefig("../Analysis Images/{}_fourier.png".format(exp_name))
    plt.close(fig)
    
    ## PHASE CALCULATION
    R = np.real(ft_data[:, 1])
    I = np.imag(ft_data[:, 1])

    # Calculating Phase
    phase = I / (R ** 2 + I ** 2) ** 0.5
    phase = sgf(phase, window_length=31, polyorder=3)

    # Normalization
    phase = phase/np.sqrt(np.sum(phase**2))

    # Plotting
    fig = plt.figure(figsize=(14,7))

    ph = fig.add_subplot(111, title="{}: Phase".format(exp_name), xlabel="time (s)")

    ph.plot(phase)

    ph.set_xticks(np.linspace(0, len(df.columns), 11))
    ph.set_xticklabels([df.columns[i] for i in range(len(df.columns)) if i%int(len(df.columns)/10) == 0])

    plt.savefig("../Analysis Images/{}_phase.png".format(exp_name))
    plt.close(fig)
    
    ## PHASE DERIVATIVE CALCULATION
    phase_deriv = np.diff(phase)
    phase_deriv = sgf(phase_deriv, window_length=31, polyorder=3)

    # Normalization
    phase_deriv = phase_deriv/np.sqrt(np.sum(phase_deriv**2))

    # Plotting
    fig = plt.figure(figsize=(14,7))
    pder = fig.add_subplot(111, title="{} Experiment Phase Derivative".format(exp_name), xlabel="time (s)")

    pder.plot(phase_deriv)

    pder.set_xticks(np.linspace(0, len(df.columns), 11))
    pder.set_xticklabels([df.columns[i] for i in range(len(df.columns)) if i%int(len(df.columns)/10) == 0])

    plt.savefig("../Analysis Images/{}_phase_deriv.png".format(exp_name))
    plt.close(fig)
    
    print("Saved all plots in ../Analysis Images/, and output {}.xlsx to ../Experiment Files/".format(output_name))
    
ser.close() # Close serial connection
c.close() # Close cursor 
conn.close() # Close connection to save memory

Experiment 2 in progress: Run 14
Experiment Name: 151
Compound: Heptane
Performing Heptane injection
9
8
7
6
5
4
3
2
1
0
9
8
7
6
5
4
3
2
1
0
9
8
7
6
5
4
3
2
1
0

------------------------------
Done Captures!
Saving to CSV in ../Scan CSV Files/151.csv
Analyzing data and saving plots.
Saved all plots in ../Analysis Images/, and output h.xlsx to ../Experiment Files/


### Experiment 3

In [ ]:
# print('Purging for 10 minutes - no measurement')
# ser.write(b'A')
# time.sleep(600)
# ser.write(b'a')

# Read excel and convert to pandas dataframe
experiment = pd.read_excel("../Experiment Files/Experiment 5.xlsx")
experiment.set_index("True Runs", inplace=True) # indexed by the True run number
experiment.dropna(axis=0, how="all", inplace=True) #removes all NaN rows

num_exp = len(experiment.index) # create an entry for each experiment run

# Indexing starts at zero
for run in range(num_exp): 
    clear_output() # clears output after every run
    
    # --------------- DESIGNING RECIPE -------------------
    
    o_time = list(experiment['Odorant Time'])[run]
    odorant_time = [o_time for i in range(3)]
    
    purge = int(list(experiment['Purge time'])[run]) # by default dataframes are floats
    
    s_time = list(experiment['Sleep Time'])[run]
    sleep_time = [s_time for i in range(3)]
    
    delay_time = 20 # scans per second

    
    # --------------- PERFORMING RECIPE -------------------
    
    # Experiment Name - Integer ID
    exp_name = "Default"
    c.execute("SELECT ExperimentID FROM Experiments")
    data = c.fetchall()
    if data == []: 
        exp_name = 1
    else:
        exp_name = data[-1][0] + 1

    # Timestamp
    timestamp = str(datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))

    # Compound name coded into its OdorID from the database
    compound = list(experiment['Current'])[run]
    temp = c.execute("SELECT OdorID FROM Odors WHERE Name='%s'" %str(compound.upper())) # All database entries are uppercase
    OdorID = c.fetchall()[0][0]
    print("Experiment 2 in progress: Run {}\nExperiment Name: {}\nCompound: {}".format(run+12, exp_name, compound))


    # Procedure name coded into its ProcedureID from the database
    temp = c.execute("SELECT ProcedureID FROM Procedures WHERE Name='Testing'")
    ProcedureID = c.fetchall()[0][0]

    # Creates a PNG and CSV File pointing to the location of the files
    PNG = "../Scan CSV Files/{}.png".format(exp_name)
    CSV = "../Scan CSV Files/PNG/{}.png".format(exp_name)


    # ======================= WRITING TO CSV File =======================

    df = pd.DataFrame() # creates a new dataframe for csv file
    df["Wavelengths"] = spec.wavelengths()
    df.set_index("Wavelengths")

    # ============================== RECIPE ==============================

    intensities = [spec.intensities()] # empty list to be populated with intensity values from spec
    elapse = [0] #empty list to be populated with time values

    compound_dict = {"Nitrogen": Nitrogen, 
                     "Water": Water, 
                     "IPA": IPA, 
                     "Ethanol": Ethanol} # Assigns the compound to its injection function
    
    print("Performing {} injection".format(compound))
    
    nitro_arg = [purge, ser, intensities, spec, elapse]
    Nitrogen(*nitro_arg)
    
    for i in range(len(odorant_time)):
        odorant_arg = [odorant_time[i], ser, intensities, spec, delay_time, elapse]
        compound_dict["{}".format(compound)](*odorant_arg) # Change the function based on what odorant you are testing
        
        sleep_arg = [sleep_time[i], ser, intensities, spec, delay_time, elapse]
        if i == (len(odorant_time)-1):
            Sleep(*sleep_arg, True)
        else: 
            Sleep(*sleep_arg)
        
        
    sleep_arg = [10, ser, intensities, spec, delay_time, elapse]
    Sleep(*sleep_arg)
    
    for i in range(len(elapse)):
        df["{}".format(round(elapse[i],2))] = intensities[i] # Write to the i'th column the intensities

    sys.stdout.write("\n{}\nDone Captures!\n".format('-'*30))

    # ========================== WRITING TO FILES ==========================
    print("Saving to CSV in ../Scan CSV Files/{}.csv".format(exp_name))
    df.to_csv("../Scan CSV Files/{}.csv".format(exp_name)) # Writing to the CSV file

    plot_river(np.array(intensities),plt, np) # Plotting river plot
    plt.savefig("../Scan CSV Files/PNG/{}.png".format(exp_name)) # Save figure to file
    plt.close()


    Notes = "Testing automation."
    Notes += "\nOdorant Times: {}\nSleep Times: {}\n".format(", ".join([str(i) for i in sleep_time]),
                                                             ", ".join([str(i) for i in odorant_time]))

    c.execute("""INSERT INTO Experiments (Timestamp, OdorID, ProcedureID, PNG, CSV, Notes)
                 VALUES (?, ?, ?, ?, ?, ?)""",
                 (timestamp, OdorID, ProcedureID, PNG, CSV, Notes))
    
    conn.commit() # Save values into SQL

    # ========================== DATA ANALYSIS ==========================
    print("Analyzing data and saving plots.")
    # Read file and convert it to pandas dataframe
    df = pd.read_csv("../Scan CSV Files/{}.csv".format(exp_name))
    df.drop("Unnamed: 0", axis=1, inplace=True)
    df.set_index("Wavelengths", inplace=True)
    exp_name = compound + "({})".format(exp_name)
    
    
    ## FOURIER TRANSFORM
    fig = plt.figure(figsize=(14,7))
    ft = fig.add_subplot(111, title="{}: Fourier transform".format(exp_name), xlabel="time (s)")

    sliced = [df.index[500], df.index[1200]] # Wavelengths between 400 and 900

    # taking the transpose means you get the fourier transform with time axis
    ft_data = np.fft.fft( (df.loc[sliced[0]:sliced[1], :]).transpose() ) 

    ft.plot(np.real(ft_data[:,1]))
    ft.plot(np.imag(ft_data[:,1]))

    # axes titles
    ft.set_xticks(np.linspace(0, len(df.columns), 11)) # 11 ticks
    ft.set_xticklabels([df.columns[i] for i in range(len(df.columns)) if i%(int(len(df.columns)/10)) == 0]) 
    plt.savefig("../Analysis Images/{}_fourier.png".format(exp_name))
    plt.close(fig)
    
    ## PHASE CALCULATION
    R = np.real(ft_data[:, 1])
    I = np.imag(ft_data[:, 1])

    # Calculating Phase
    phase = I / (R ** 2 + I ** 2) ** 0.5
    phase = sgf(phase, window_length=31, polyorder=3)

    # Normalization
    phase = phase/np.sqrt(np.sum(phase**2))

    # Plotting
    fig = plt.figure(figsize=(14,7))

    ph = fig.add_subplot(111, title="{}: Phase".format(exp_name), xlabel="time (s)")

    ph.plot(phase)

    ph.set_xticks(np.linspace(0, len(df.columns), 11))
    ph.set_xticklabels([df.columns[i] for i in range(len(df.columns)) if i%int(len(df.columns)/10) == 0])

    plt.savefig("../Analysis Images/{}_phase.png".format(exp_name))
    plt.close(fig)
    
    ## PHASE DERIVATIVE CALCULATION
    phase_deriv = np.diff(phase)
    phase_deriv = sgf(phase_deriv, window_length=31, polyorder=3)

    # Normalization
    phase_deriv = phase_deriv/np.sqrt(np.sum(phase_deriv**2))

    # Plotting
    fig = plt.figure(figsize=(14,7))
    pder = fig.add_subplot(111, title="{} Experiment Phase Derivative".format(exp_name), xlabel="time (s)")

    pder.plot(phase_deriv)

    pder.set_xticks(np.linspace(0, len(df.columns), 11))
    pder.set_xticklabels([df.columns[i] for i in range(len(df.columns)) if i%int(len(df.columns)/10) == 0])

    plt.savefig("../Analysis Images/{}_phase_deriv.png".format(exp_name))
    plt.close(fig)
    
    print("Saved all plots in ../Analysis Images/, and output {}.xlsx to ../Experiment Files/".format(output_name))
    
ser.close() # Close serial connection
c.close() # Close cursor 
conn.close() # Close connection to save memory

In [ ]:
import time
import sys 

start_time = time.time()
i = 0 

while i < 20: 
    if round(i, 1) in [j for j in range(20)]: 
        print(round(i), end="\r")
    time.sleep(1/20)
    i = time.time() - start_time
    

### Testing Procedure

In [ ]:
# Allow user to choose the experiment file
root = Tk()
root.fileName = filedialog.askopenfilename( filetypes = ( ("Excel File", "*.xlsx"), ("All Files", "*.*") ) )

# Read excel and convert to pandas dataframe
experiment = pd.read_excel('{}'.format(root.fileName))
experiment.set_index("True Runs", inplace=True) # indexed by the True run number
experiment.dropna(axis=0, how="all", inplace=True) #removes all NaN rows

# Set output to excel file (6 response variables)
to_df = {"Run": [], 
         "Steady State": [], 
         "Peak 1": [], 
         "Peak 2": [], 
         "Peak 3": [], 
         "Trough 1": [], 
         "Trough 2": []}

df2 = pd.DataFrame(to_df)
df2.set_index('Run', inplace=True)
df2.to_excel('../Experiment Files/{}.xlsx'.format(output_name))

num_exp = len(experiment.index) # create an entry for each experiment run

ser.write(b'A')
time.sleep(600)
ser.write(b'a')

# Indexing starts at zero
for run in range(num_exp): 
    clear_output() # clears output after every run
    
    # --------------- DESIGNING RECIPE -------------------
    
    o_time = list(experiment['Odorant Time'])[run]
    odorant_time = [o_time for i in range(3)]
    
    purge = int(list(experiment['Purge time'])[run]) # by default dataframes are floats
    
    c_time = list(experiment['Clean Time'])[run]
    clean_time = [purge] + [c_time for i in range(2)]
    
    delay_time = 10 # scans per second

    
    # --------------- PERFORMING RECIPE -------------------
    
    # Experiment Name - Integer ID
    exp_name = "Default"
    c.execute("SELECT ExperimentID FROM Experiments")
    data = c.fetchall()
    if data == []: 
        exp_name = 1
    else:
        exp_name = data[-1][0] + 1

    # Timestamp
    timestamp = str(datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))

    # Compound name coded into its OdorID from the database
    compound = list(experiment['Current'])[run]
    temp = c.execute("SELECT OdorID FROM Odors WHERE Name='%s'" %str(compound.upper())) # All database entries are uppercase
    OdorID = c.fetchall()[0][0]
    print("Run Number: {}\nExperiment Name: {}\nCompound: {}".format(run+1, exp_name, compound))


    # Procedure name coded into its ProcedureID from the database
    temp = c.execute("SELECT ProcedureID FROM Procedures WHERE Name='Testing'")
    ProcedureID = c.fetchall()[0][0]

    # Creates a PNG and CSV File pointing to the location of the files
    PNG = "../Scan CSV Files/{}.png".format(exp_name)
    CSV = "../Scan CSV Files/PNG/{}.png".format(exp_name)


    # ======================= WRITING TO CSV File =======================

    df = pd.DataFrame() # creates a new dataframe for csv file
    df["Wavelengths"] = spec.wavelengths()
    df.set_index("Wavelengths")

    # ============================== RECIPE ==============================

    intensities = [spec.intensities()] # empty list to be populated with intensity values from spec
    elapse = [0] #empty list to be populated with time values

    compound_dict = {"Nitrogen": Nitrogen, 
                     "Water": valveC, 
                     "IPA": valveD, 
                     "Ethanol": valveB,
                     "Pentane": valveC,
                     "Hexane": valveD, 
                     "Heptane": valveB} # Assigns the compound to its injection function

    # If there is a nitrogen flush specified beforehand, then do a 10 minute flush.
    print("Purging chamber for 1 minute. Please standby.")
    Purge(60, ser) # Purge chamber with both nitrogen and ethanol
    
    print("Performing {} injection".format(compound))
    for i in range(len(odorant_time)):
        nitro_arg = [clean_time[i], ser, intensities, spec, elapse]
        Nitrogen(*nitro_arg)

        odorant_arg = [odorant_time[i], ser, intensities, spec, delay_time, elapse]
        compound_dict["{}".format(compound)](*odorant_arg) # Change the function based on what odorant you are testing
    
    for i in range(len(elapse)):
        df["{}".format(round(elapse[i],2))] = intensities[i] # Write to the i'th column the intensities

    sys.stdout.write("\n{}\nDone Captures!\n".format('-'*30))

    # ========================== WRITING TO FILES ==========================
    print("Saving to CSV in ../Scan CSV Files/{}.csv".format(exp_name))
    df.to_csv("../Scan CSV Files/{}.csv".format(exp_name)) # Writing to the CSV file

    plot_river(np.array(intensities),plt, np) # Plotting river plot
    plt.savefig("../Scan CSV Files/PNG/{}.png".format(exp_name)) # Save figure to file
    plt.close()


    Notes = "Testing automation."
    Notes += "\nOdorant Times: {}\nClean Times: {}\n".format(", ".join([str(i) for i in clean_time]),
                                                             ", ".join([str(i) for i in odorant_time]))

    c.execute("""INSERT INTO Experiments (Timestamp, OdorID, ProcedureID, PNG, CSV, Notes)
                 VALUES (?, ?, ?, ?, ?, ?)""",
                 (timestamp, OdorID, ProcedureID, PNG, CSV, Notes))
    
    conn.commit() # Save values into SQL

    # ========================== DATA ANALYSIS ==========================
    print("Analyzing data and saving plots.")
    # Read file and convert it to pandas dataframe
    df = pd.read_csv("../Scan CSV Files/{}.csv".format(exp_name))
    df.drop("Unnamed: 0", axis=1, inplace=True)
    df.set_index("Wavelengths", inplace=True)
    exp_name = compound + "({})".format(exp_name)
    
    
    ## FOURIER TRANSFORM
    fig = plt.figure(figsize=(14,7))
    ft = fig.add_subplot(111, title="{}: Fourier transform".format(exp_name), xlabel="time (s)")

    sliced = [df.index[500], df.index[1200]] # Wavelengths between 400 and 900

    # taking the transpose means you get the fourier transform with time axis
    ft_data = np.fft.fft( (df.loc[sliced[0]:sliced[1], :]).transpose() ) 

    ft.plot(np.real(ft_data[:,1]))
    ft.plot(np.imag(ft_data[:,1]))

    # axes titles
    ft.set_xticks(np.linspace(0, len(df.columns), 11)) # 11 ticks
    ft.set_xticklabels([df.columns[i] for i in range(len(df.columns)) if i%(int(len(df.columns)/10)) == 0]) 
    plt.savefig("../Analysis Images/{}_fourier.png".format(exp_name))
    plt.close(fig)
    
    ## PHASE CALCULATION
    R = np.real(ft_data[:, 1])
    I = np.imag(ft_data[:, 1])

    # Calculating Phase
    phase = I / (R ** 2 + I ** 2) ** 0.5
    phase = sgf(phase, window_length=31, polyorder=3)

    # Normalization
    phase = phase/np.sqrt(np.sum(phase**2))

    # Plotting
    fig = plt.figure(figsize=(14,7))

    ph = fig.add_subplot(111, title="{}: Phase".format(exp_name), xlabel="time (s)")

    ph.plot(phase)

    ph.set_xticks(np.linspace(0, len(df.columns), 11))
    ph.set_xticklabels([df.columns[i] for i in range(len(df.columns)) if i%int(len(df.columns)/10) == 0])

    plt.savefig("../Analysis Images/{}_phase.png".format(exp_name))
    plt.close(fig)
    
    ## PHASE DERIVATIVE CALCULATION
    phase_deriv = np.diff(phase)
    phase_deriv = sgf(phase_deriv, window_length=31, polyorder=3)

    # Normalization
    phase_deriv = phase_deriv/np.sqrt(np.sum(phase_deriv**2))

    # Plotting
    fig = plt.figure(figsize=(14,7))
    pder = fig.add_subplot(111, title="{} Experiment Phase Derivative".format(exp_name), xlabel="time (s)")

    pder.plot(phase_deriv)

    pder.set_xticks(np.linspace(0, len(df.columns), 11))
    pder.set_xticklabels([df.columns[i] for i in range(len(df.columns)) if i%int(len(df.columns)/10) == 0])

    plt.savefig("../Analysis Images/{}_phase_deriv.png".format(exp_name))
    plt.close(fig)
    
    
    # ========================== RESPONSE ACQUISITION ==========================
    
    ## STEADY STATE
    def steady_state(seq, n=60, eps=0.2):
        seq = abs(seq) < eps
        for i in range(len(seq)):
            win = seq[i:i+n]
            if sum(win) == n:
                return int(i+n/2)
        return 0

    ss_pd = phase_deriv[:purge-20] # Use only the part of the spectrum when nitrogen is injected
    
    # Use half of one standard deviation as the steady state height condition (only applies to purge time)
    eps = min([np.mean(ss_pd) + np.std(ss_pd), np.mean(ss_pd) + np.std(ss_pd)])

    threshold = steady_state(ss_pd, eps=eps)

    ## PEAKS AND TROUGHS
    def window(ind, peak_index, n=60):
        for i in range(ind-n, ind+n): 
            if i in peak_index: 
                return False
        return True

    ss_pd2 = phase_deriv[purge-20:] # Use only up until first odorant is injected
    eps2 = max([np.mean(ss_pd2) + np.std(ss_pd2), np.mean(ss_pd2) + np.std(ss_pd2)]) # take standard deviation of odorant part

    peak_list = list(zip(peaks(ss_pd2, eps2)[1]['peak_heights'], peaks(ss_pd2, eps2)[0])) # list of tuples with heights and indexes
    peak_list_sort = sorted(peak_list, reverse=True) # sort list from highest peak
    peak_index = [] # populate with local maxima

    for i in peak_list_sort:
        if window(i[1], peak_index): # if there are multiple peaks around the same point, choose the highest one
            peak_index.append(+i[1])
    peak_height = [i[0] for i in peak_list if i[1] in peak_index[:3]] # assign peak heights in order
    peak_index = [purge-20+i for i in peak_index[:3]] # pick only the highest three local maxima

    trough_list = list(zip(peaks(-ss_pd2, eps2)[1]['peak_heights'], peaks(-ss_pd2, eps2)[0])) # invert curve and treat troughs like peaks
    trough_list_sort = sorted(trough_list, reverse=True)
    trough_index = []

    for i in trough_list_sort:
        if window(i[1], trough_index): # if there are multiple peaks around the same point, choose the highest one
            trough_index.append(+i[1])

    trough_height = [i[0] for i in trough_list if i[1] in trough_index[:2]] # assign peak heights in order
    trough_index = [purge-20+i for i in trough_index[:2]]

    fig = plt.figure(figsize=(14, 7))
    pder = fig.add_subplot(111, title="{} Experiment Phase Derivative".format(exp_name), 
                           xlabel="time (s)", 
                           ylabel="Absolute Difference")

    dfcols = [float(i) for i in df.columns[:-1]] # we lose a single value by taking derivative of phase
    pder.plot(dfcols, phase_deriv)
    for i in peak_index: 
        pder.plot(float(df.columns[i]), phase_deriv[i], 'r*')
    for i in trough_index:
        pder.plot(float(df.columns[i]), phase_deriv[i], 'g*')
    pder.plot(float(df.columns[threshold]), phase_deriv[threshold], 'k*')

    plt.axvspan(float(df.columns[threshold-30]),float(df.columns[threshold+30]), color="y", alpha=0.2) # plot steady state window
    plt.axhspan(-eps,eps, color="b", alpha=0.1) # plot steady state std window
    plt.axhspan(-eps2,eps2, color="r", alpha=0.1) # plot peak std window
    plt.vlines(float(df.columns[threshold]),min(phase_deriv), max(phase_deriv), alpha=0.5, linestyles={'dashed'}, label='{}'
               .format((float(df.columns[threshold]), round(phase_deriv[threshold],2))))
    plt.legend()

    plt.savefig("../Analysis Images/{}_SS_Peaks.png".format(exp_name))
    plt.close(fig)
    
    # -------------------------- SAVE RESPONSE VARIABLES TO SPREADSHEET -------------------------------
    to_df['Run'].append(run)
    try: 
        to_df['Steady State'].append(threshold)
        to_df['Peak 1'].append(peak_height[0])
        to_df['Peak 2'].append(peak_height[1])
        to_df['Peak 3'].append(peak_height[2])
        to_df['Trough 1'].append(trough_height[0])
        to_df['Trough 2'].append(trough_height[1])
    except IndexError:
        to_df['Peak 1'].append(0)
        to_df['Peak 2'].append(0)
        to_df['Peak 3'].append(0)
        to_df['Trough 1'].append(0)
        to_df['Trough 2'].append(0)
    
    df2 = pd.DataFrame(to_df)
    df2.set_index('Run', inplace=True)
    df2.head()
    df2.to_excel('../Experiment Files/{}.xlsx'.format(output_name))
    
    print("Saved all plots in ../Analysis Images/, and output {}.xlsx to ../Experiment Files/".format(output_name))
    
ser.close() # Close serial connection
c.close() # Close cursor 
conn.close() # Close connection to save memory

In [ ]:
# import time
# import serial
# ser = serial.Serial("COM4", 9600)
ser.write(b'abcd')
# ser.close()